In [17]:
# Trimming genome msa files to only extract RBD segment and remove gaps to get true RBD nucleotide sequences

def extract_sequences_in_range(msa_fasta_file, start_position, end_position, output_file):
    with open(msa_fasta_file, "r") as file:
        header = ""
        sequence = ""

        # Open the output file for writing
        with open(output_file, "w") as output_file:

            for line in file:
                line = line.strip()

                if line.startswith(">"):
                    # Start a new sequence
                    if header and sequence:
                        # Process the previous sequence if it's complete
                        sequence = sequence.replace("*", "")
                        extracted_sequence = sequence[start_position - 1:end_position]

                        # Remove gaps from the extracted sequence.
                        extracted_sequence = extracted_sequence.replace("-", "")

                        # Write the extracted sequence to the output file.
                        output_file.write(f">{header}\n{extracted_sequence}\n")

                    header = line
                    sequence = ""
                else:
                    sequence += line

            # Process the last sequence if it's complete
            if header and sequence:
                sequence = sequence.replace("*", "")
                extracted_sequence = sequence[start_position - 1:end_position]
                extracted_sequence = extracted_sequence.replace("-", "")
                output_file.write(f">{header}\n{extracted_sequence}\n")

    print(f"Extracted sequences between positions {start_position} and {end_position} to '{output_file}'.")

def main():
    msa_fasta_file = "your/path/ref_genome_MSA.fasta"
    start_position = 34732
    end_position = 36397
    output_file = "your/path/ref_genome_RBD_nucleotides.fasta"
    extract_sequences_in_range(msa_fasta_file, start_position, end_position, output_file)

if __name__ == "__main__":
    main()


Extracted sequences between positions 34732 and 36397 to '<_io.TextIOWrapper name='ref_genome_RBD_nucleotides.fasta' mode='w' encoding='UTF-8'>'.


In [1]:
# Length Statistics for trimmed reads
from Bio import SeqIO

def generate_sequence_statistics(fasta_file):
    sequence_lengths = []
    total_sequences = 0
    total_bases = 0

    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            total_sequences += 1
            sequence_length = len(record.seq)
            sequence_lengths.append(sequence_length)
            total_bases += sequence_length

    # Calculate statistics
    min_length = min(sequence_lengths)
    max_length = max(sequence_lengths)
    average_length = total_bases / total_sequences

    # Print the statistics
    print("Sequence Statistics:")
    print(f"Total Sequences: {total_sequences}")
    print(f"Total Bases: {total_bases}")
    print(f"Minimum Length: {min_length} bases")
    print(f"Maximum Length: {max_length} bases")
    print(f"Average Length: {average_length:.2f} bases")

if __name__ == "__main__":
    fasta_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/msaCodon_1024_trimmed_RBD_new.fasta"  # Replace with your FASTA file
    generate_sequence_statistics(fasta_file)


Sequence Statistics:
Total Sequences: 1660854
Total Bases: 1114385340
Minimum Length: 0 bases
Maximum Length: 1600 bases
Average Length: 670.97 bases


In [2]:
# Display number of sequences in fasta file with all possible readlengths 
from collections import defaultdict

def count_sequence_lengths(fasta_file):
    sequence_lengths = defaultdict(int)
    
    with open(fasta_file, "r") as file:
        sequence = ""
        
        for line in file:
            if line.startswith(">"):
                if sequence:
                    sequence_lengths[len(sequence)] += 1
                sequence = ""
            else:
                sequence += line.strip()
        
        # Don't forget to count the last sequence
        if sequence:
            sequence_lengths[len(sequence)] += 1
    
    return sequence_lengths

def print_sequence_lengths(fasta_file):
    sequence_lengths = count_sequence_lengths(fasta_file)
    
    for length, count in sorted(sequence_lengths.items()):
        print(f"Read length: {length}, Number of sequences: {count}")

def main():
    fasta_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/msaCodon_1024_trimmed_RBD_new.fasta"
    print_sequence_lengths(fasta_file)

if __name__ == "__main__":
    main()


Read length: 358, Number of sequences: 1
Read length: 380, Number of sequences: 1
Read length: 410, Number of sequences: 1
Read length: 413, Number of sequences: 4
Read length: 417, Number of sequences: 1
Read length: 439, Number of sequences: 1
Read length: 440, Number of sequences: 1
Read length: 441, Number of sequences: 2
Read length: 442, Number of sequences: 1
Read length: 446, Number of sequences: 1
Read length: 452, Number of sequences: 1
Read length: 470, Number of sequences: 1
Read length: 476, Number of sequences: 1
Read length: 482, Number of sequences: 1
Read length: 485, Number of sequences: 3
Read length: 489, Number of sequences: 1
Read length: 493, Number of sequences: 1
Read length: 495, Number of sequences: 1
Read length: 501, Number of sequences: 1
Read length: 502, Number of sequences: 1
Read length: 510, Number of sequences: 2
Read length: 511, Number of sequences: 1
Read length: 519, Number of sequences: 2
Read length: 526, Number of sequences: 1
Read length: 527

In [3]:
# Calculate first quartile (Q1), third quartile (Q3), and the interquartile range (IQR) and 
# use the IQR to determine the lower and upper bounds for outlier detection and 
# remove outlier readlengths.

import numpy as np
from collections import defaultdict

def count_sequence_lengths(fasta_file):
    sequence_lengths = []
    sequences = []

    with open(fasta_file, "r") as file:
        sequence = ""
        header = ""

        for line in file:
            if line.startswith(">"):
                if sequence:
                    sequence_lengths.append(len(sequence))
                    sequences.append((header, sequence))
                header = line.strip()
                sequence = ""
            else:
                sequence += line.strip()

        # Don't forget to count the last sequence
        if sequence:
            sequence_lengths.append(len(sequence))
            sequences.append((header, sequence))

    return sequence_lengths, sequences

def calculate_iqr_outliers(sequence_lengths):
    q1 = np.percentile(sequence_lengths, 25)
    q3 = np.percentile(sequence_lengths, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return lower_bound, upper_bound

def filter_outliers_and_write(sequences, lower_bound, upper_bound, output_file):
    with open(output_file, "w") as outfile:
        for header, sequence in sequences:
            if lower_bound <= len(sequence) <= upper_bound:
                outfile.write(f"{header}\n{sequence}\n")

def main():
    fasta_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/msaCodon_1024_trimmed_RBD_new.fasta"
    output_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new.fasta"

    sequence_lengths, sequences = count_sequence_lengths(fasta_file)
    lower_bound, upper_bound = calculate_iqr_outliers(sequence_lengths)
    filter_outliers_and_write(sequences, lower_bound, upper_bound, output_file)

    print(f"Sequences between lengths {int(lower_bound)} and {int(upper_bound)} written to '{output_file}'.")

if __name__ == "__main__":
    main()


Sequences between lengths 671 and 671 written to '/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new.fasta'.


In [4]:
# Length Statistics for filtered trimmed reads
from Bio import SeqIO

def generate_sequence_statistics(fasta_file):
    sequence_lengths = []
    total_sequences = 0
    total_bases = 0

    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            total_sequences += 1
            sequence_length = len(record.seq)
            sequence_lengths.append(sequence_length)
            total_bases += sequence_length

    # Calculate statistics
    min_length = min(sequence_lengths)
    max_length = max(sequence_lengths)
    average_length = total_bases / total_sequences

    # Print the statistics
    print("Sequence Statistics:")
    print(f"Total Sequences: {total_sequences}")
    print(f"Total Bases: {total_bases}")
    print(f"Minimum Length: {min_length} bases")
    print(f"Maximum Length: {max_length} bases")
    print(f"Average Length: {average_length:.2f} bases")

if __name__ == "__main__":
    fasta_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_new.fasta"  # Replace with your FASTA file
    generate_sequence_statistics(fasta_file)


Sequence Statistics:
Total Sequences: 1659465
Total Bases: 1113501015
Minimum Length: 671 bases
Maximum Length: 671 bases
Average Length: 671.00 bases


In [6]:
## Python script to make a subset of the filtered_msaCodon_1024_trimmed_RBD.fasta without N 
from Bio import SeqIO
import random

# Input FASTA file and output file
input_fasta = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new.fasta"
output_fasta = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new_1_6mil.fasta"

# Read all sequences from the input FASTA file
all_sequences = list(SeqIO.parse(input_fasta, "fasta"))

# Randomly select 1 million sequences
#num_sequences_to_select = 1600000
num_sequences_to_select = 3000000

subset_sequences = random.sample(all_sequences, min(num_sequences_to_select, len(all_sequences)))

# Filter sequences to contain only A, T, G, and C, and have lengths as multiples of 3
valid_sequences = [seq for seq in subset_sequences if set(str(seq.seq)).issubset("ATGC")]

# Ensure each selected sequence length is a multiple of 3 or remove 1 or 2 nucleotides
for i in range(len(valid_sequences)):
    seq_len = len(valid_sequences[i])
    if seq_len % 3 != 0:
        trim_length = seq_len % 3
        valid_sequences[i] = valid_sequences[i][:-trim_length]

# Write the selected sequences to the output FASTA file
with open(output_fasta, "w") as output_handle:
    SeqIO.write(valid_sequences, output_handle, "fasta")

print(f"Randomly selected {num_sequences_to_select} valid sequences and saved to {output_fasta}")


Randomly selected 1600000 valid sequences and saved to /mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new_1_6mil.fasta


In [7]:
# Length Statistics for subset of filtered trimmed reads
from Bio import SeqIO

def generate_sequence_statistics(fasta_file):
    sequence_lengths = []
    total_sequences = 0
    total_bases = 0

    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            total_sequences += 1
            sequence_length = len(record.seq)
            sequence_lengths.append(sequence_length)
            total_bases += sequence_length

    # Calculate statistics
    min_length = min(sequence_lengths)
    max_length = max(sequence_lengths)
    average_length = total_bases / total_sequences

    # Print the statistics
    print("Sequence Statistics:")
    print(f"Total Sequences: {total_sequences}")
    print(f"Total Bases: {total_bases}")
    print(f"Minimum Length: {min_length} bases")
    print(f"Maximum Length: {max_length} bases")
    print(f"Average Length: {average_length:.2f} bases")

if __name__ == "__main__":
    fasta_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new_1_6mil.fasta"  # Replace with your FASTA file
    generate_sequence_statistics(fasta_file)

Sequence Statistics:
Total Sequences: 1600000
Total Bases: 1070400000
Minimum Length: 669 bases
Maximum Length: 669 bases
Average Length: 669.00 bases


In [5]:
# Length Statistics for filtered trimmed reads
from Bio import SeqIO

def generate_sequence_statistics(fasta_file):
    sequence_lengths = []
    total_sequences = 0
    total_bases = 0

    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            total_sequences += 1
            sequence_length = len(record.seq)
            sequence_lengths.append(sequence_length)
            total_bases += sequence_length

    # Calculate statistics
    min_length = min(sequence_lengths)
    max_length = max(sequence_lengths)
    average_length = total_bases / total_sequences

    # Print the statistics
    print("Sequence Statistics:")
    print(f"Total Sequences: {total_sequences}")
    print(f"Total Bases: {total_bases}")
    print(f"Minimum Length: {min_length} bases")
    print(f"Maximum Length: {max_length} bases")
    print(f"Average Length: {average_length:.2f} bases")

if __name__ == "__main__":
    fasta_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil.fasta"  # Replace with your FASTA file
    generate_sequence_statistics(fasta_file)


Sequence Statistics:
Total Sequences: 3000000
Total Bases: 2007000000
Minimum Length: 669 bases
Maximum Length: 669 bases
Average Length: 669.00 bases


In [8]:
# Script to write out matching fasta files with Accession ID values in metadata file to csv 
# files with Variant values from voc.json file and convert each nucleotide sequence to protein sequences

from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import json
from tqdm import tqdm
from multiprocessing import Pool, cpu_count, Manager

# Load your metadata DataFrame (df_metadata) and provide the correct path to your FASTA file and "RBD.voc.json" file.
csv_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/metadata_2024_07_09.tsv"
df_metadata = pd.read_csv(csv_file, sep='\t', dtype=str, low_memory=False, encoding='latin-1')

# Load the voc.json file
with open('/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/voc.json', 'r') as json_file:
    voc_data = json.load(json_file)

# Define a function to determine the VOC value based on Pango Lineage
def determine_voc(lineage):
    for voc, lineages in voc_data.items():
        if lineage in lineages:
            return voc
    return "nonVOC"

# Define the translation function
def translate_nucleotides_to_protein(nucleotide_sequence):
    return str(Seq(nucleotide_sequence).translate())

# Define the output CSV file
output_csv_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new_1_6mil.csv"

# Initialize a count for matches
match_count = 0
total_sequences = 0

# Function to process each sequence
def process_sequence(record):
    global df_metadata, voc_data
    accession_id_fasta = record.description.split("|")[1]
    sequence = str(record.seq)

    # Check if the Accession ID is present in the metadata DataFrame
    match_row = df_metadata[df_metadata['Accession ID'] == accession_id_fasta]
    if not match_row.empty:
        lineage = match_row['Pango lineage'].values[0]

        # Create a dictionary for the current record
        output_data = {
            'Accession ID': accession_id_fasta,
            'Lineage': lineage,
            'RBD nucleotide': sequence,
            'Variant': determine_voc(lineage),
            'RBD protein': translate_nucleotides_to_protein(sequence),
        }
        return output_data
    return None

# Write the header to the output CSV file
with open(output_csv_file, 'w', newline='') as f:
    writer = pd.DataFrame(columns=['Accession ID', 'Lineage', 'RBD nucleotide', 'Variant', 'RBD protein'])
    writer.to_csv(f, index=False)

# Estimate the total number of sequences for tqdm
with open("/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new_1_6mil.fasta", "r") as fasta_file:
    total_sequences_estimated = sum(1 for line in fasta_file if line.startswith(">"))

# Process sequences using multiprocessing Pool
with Pool(processes = 32) as pool, tqdm(total=total_sequences_estimated, desc="Processing sequences") as pbar:
    for result in pool.imap(process_sequence, SeqIO.parse("/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new_1_6mil.fasta", "fasta")):
        pbar.update()
        if result:
            with open(output_csv_file, 'a', newline='') as f:
                output_df = pd.DataFrame([result])
                output_df.to_csv(f, mode='a', header=False, index=False)
                match_count += 1

# Print the results
print(f"Matches found: {match_count}")
print(f"Total sequences: {total_sequences_estimated}")
print(f"Output CSV file saved as {output_csv_file}")


Processing sequences: 100%|██████████| 1600000/1600000 [15:45:45<00:00, 28.20it/s]   


Matches found: 1599926
Total sequences: 1600000
Output CSV file saved as /mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_new_1_6mil.csv


In [8]:
# Script to write out matching fasta files with Accession ID values in metadata file to csv 
# files with Variant values directly from metadata file 
# And convert each nucleotide sequence to protein sequences

from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import csv
from tqdm import tqdm

# Load your metadata DataFrame and create a dictionary for quick lookups
csv_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/metadata_2024_07_09.tsv"
df_metadata = pd.read_csv(csv_file, sep='\t', dtype=str, low_memory=False, encoding='latin-1')

# Create a dictionary with Accession ID as the key and (Variant, Pango lineage) as the value
metadata_dict = df_metadata.set_index('Accession ID')[['Variant', 'Pango lineage']].to_dict('index')

# Define the translation function
def translate_nucleotides_to_protein(nucleotide_sequence):
    return str(Seq(nucleotide_sequence).translate())

# Define the output CSV file
output_csv_file = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil.csv"

# Estimate the total number of sequences for tqdm
with open("/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil.fasta", "r") as fasta_file:
    total_sequences_estimated = sum(1 for line in fasta_file if line.startswith(">"))

# Open the output CSV file for writing and write the header
with open(output_csv_file, 'w', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['Accession ID', 'Lineage', 'RBD nucleotide', 'Variant', 'RBD protein'])

    # Process sequences
    with tqdm(total=total_sequences_estimated, desc="Processing sequences") as pbar:
        for record in SeqIO.parse("/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil.fasta", "fasta"):
            accession_id_fasta = record.description.split("|")[1]
            sequence = str(record.seq)

            # Check if the Accession ID is present in the metadata dictionary
            if accession_id_fasta in metadata_dict:
                metadata = metadata_dict[accession_id_fasta]
                variant = metadata['Variant']
                lineage = metadata['Pango lineage']

                # Write the sequence data to the output CSV file
                writer.writerow([
                    accession_id_fasta,
                    lineage,
                    sequence,
                    variant,
                    translate_nucleotides_to_protein(sequence)
                ])
            pbar.update(1)

# Print the results
print(f"Output CSV file saved as {output_csv_file}")

Processing sequences: 100%|██████████| 3000000/3000000 [05:05<00:00, 9805.97it/s] 

Output CSV file saved as /mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil.csv


In [10]:
# Mapping Variant label from input csv to respective recognizable VOC labels
import pandas as pd
from tqdm import tqdm

# Path to the input and output CSV files
input_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil.csv"
output_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil_with_VOC.csv"

# Mapping of keywords to VOC values
variant_to_voc_mapping = {
    'wuhanh1': 'WuhanHu1',
    'delta': 'delta',
    'omicron': 'omicron',
    'alpha': 'alpha',
    'beta': 'beta',
    'epsilon': 'epsilon',
    'eta': 'eta',
    'iota': 'iota',
    'kappa': 'kappa',
    'mu': 'mu',
    'lambda': 'lambda',
    'gamma': 'gamma',
    'zeta': 'zeta',
}

# Function to map Variant to VOC
def map_variant_to_voc(variant):
    if isinstance(variant, str):  # Check if the variant is a string
        variant_lower = variant.lower()  # Convert to lowercase for case-insensitive matching
        for keyword, voc_value in variant_to_voc_mapping.items():
            if keyword in variant_lower:
                return voc_value
    return "nonVOC"  # Return a default value if no keyword matches or if variant is not a string

# Load the input CSV file
df = pd.read_csv(input_csv_path)

# Apply the mapping function to create a new 'VOC' column
df['VOC'] = df['Variant'].apply(map_variant_to_voc)

# Save the updated DataFrame to a new CSV file
df.to_csv(output_csv_path, index=False)

print(f"Updated CSV file saved as {output_csv_path}")

Updated CSV file saved as /mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/intermediate_files/filtered_msaCodon_1024_trimmed_RBD_3mil_with_VOC.csv
